In [ ]:
import numpy as np

# np.random.seed(0) # REMOVE FOR FINAL SOLUTION

In [ ]:
def splitData(data):
    shuffled = data.copy()
    np.random.shuffle(shuffled)

    testCount = int(shuffled.shape[0]*1/3)
    trainCount = shuffled.shape[0]-testCount
    trainSet = shuffled[0:trainCount]
    testSet = shuffled[trainCount:]

    return trainSet, testSet

def separateInputOutput(dataset):
    outputData = dataset[:,3] # Output
    inputData = np.concatenate((dataset[:,0:3],dataset[:,4:7],dataset[:,8:10]),axis=1)
    return inputData, outputData

def getInputAttributeDict():
    return {   0 : 'calorific_value',
                    1 : 'nitrogen',
                    2 : 'turbidity',
                    3 : 'alcohol',
                    4 : 'sugars',
                    5 : 'bitterness',
                    6 : 'colour',
                    7 : 'degree_of_fermentation'} .copy()

#ML ALG
def initializeWeights(x,y):
    N = x.shape[0]
    
def normaliseData(dataArray, normType = "range"):
    normalisedDataList = []

    if normType == "range":
        maxVal = dataArray.max()
        minVal = dataArray.min()
        for val in dataArray:
            normalisedDataList.append((val-minVal)/(maxVal-minVal))
        return np.array(normalisedDataList)

    elif normType == "z":
        mean = dataArray.mean()
        stdDev =  getStandardDeviation(dataArray)
        for val in dataArray:
            normalisedDataList.append((val-mean)/stdDev)
        return np.array(normalisedDataList)

def getStandardDeviation(dataArray):
    n = dataArray.shape[0]
    mean = dataArray.mean()
    sumOfSquareDiffs = 0
    for val in dataArray:
        sumOfSquareDiffs += (val - mean)**2

    variance = sumOfSquareDiffs/(n-1)
    stdDev = variance**(1/2)
    return stdDev


In [ ]:
# Read in data:
with open("beer.txt", 'r') as f:
    lines = np.asarray(f.read().split('\n'))

dataset = []
# Split columns and convert numbers from string to float
for lineStr in lines: 
    attributesStr = lineStr.split('\t') # Separate attributes (currently all strings)
    sample = np.empty((len(attributesStr))).astype(object) # Create empty object array for sample data as floats and str
    for i, string in enumerate(attributesStr):
        try:
            sample[i] = float(string)
        except ValueError:
            sample[i] = string
    dataset.append(sample)
dataset = np.asarray(dataset)


trainSet, testSet = splitData(dataset)
print(trainSet[0,7], trainSet[-1,7], testSet[0,7], testSet[-1,7])


testIn, testOut = separateInputOutput(testSet)
trainIn, trainOut = separateInputOutput(trainSet)

N =trainIn.shape[0]
# Wi = 1/N
# print(getInputAttributeDict()[1])



In [ ]:
def getUniqueClassCount(listToCount):
    classes = []
    for item in listToCount:
        if item not in classes:
            classes.append(item)

    counts = []
    for itemClass in classes:
        counts.append(listToCount.count(itemClass))
    return counts, classes

def getLeafGini(counts):
    total = np.sum(counts)
    giniLeaf = 1
    for count in counts:
        giniLeaf -= (count/total)**2
    return giniLeaf

def getNodeGini(sortedList, threshold):
        lessThan = []
        greaterThan = []

        for i in range(len(sortedList)):
            x = sortedList[i][0]
            # print(x, threshold)
            if x < threshold:
                lessThan.append(sortedList[i][1])
            else:
                greaterThan.append(sortedList[i][1])

        countsLT, _ = getUniqueClassCount(lessThan)
        countsGT, _ = getUniqueClassCount(greaterThan)
        giniLT = getLeafGini(countsLT)
        giniGT = getLeafGini(countsGT)
        # print(giniLT,giniGT)
        # print(np.sum(countsLT),np.sum(countsGT))
        totalCount = len(sortedList)
        # print(totalCount, (len(lessThan)+len(greaterThan)))
        nodeGini = ((np.sum(countsLT)/totalCount)*giniLT) + ((np.sum(countsGT)/totalCount)*giniGT)
        # print(giniNode)
        return nodeGini


In [ ]:
def getMinGiniAndThreshold(inputData, outputData, alreadyUsedAttributes):
    inputAttributesMinGinis = []
    attributeThresholds = []
    attributeIndexes = list(getInputAttributeDict())
    # print(attributeIndexes)
    for i in alreadyUsedAttributes:
        attributeIndexes.remove(i)
    # print("Getting gini of attributes {}".format(attributeIndexes))
    
    for i in attributeIndexes:
        attribute = inputData[:, i].copy() # Single attribute from input data
        
        inputOutputPairs = []
        for j,sample in enumerate(attribute):
            inputOutputPairs.append((sample,outputData[j]))
        attribute.sort()

        sortedInputOutputPairs = [tuple for x in attribute for tuple in inputOutputPairs if tuple[0] == x]
        # print(len(sortedInputOutputPairs), len(attribute))
        # print(attribute[-1], sortedInputOutputPairs[-1])
        testThresholds = []
        for j in range(len(attribute)-1):
            testThresholds.append((attribute[j]+attribute[j+1])/2)

        nodeGinis = []
        for testThresh in testThresholds:
            nodeGinis.append(getNodeGini(sortedInputOutputPairs, testThresh))

        minNodeGini = min(nodeGinis)
        minGiniIndex = nodeGinis.index(minNodeGini)
        attributeThreshold = testThresholds[minGiniIndex]

        inputAttributesMinGinis.append(minNodeGini)
        attributeThresholds.append(attributeThreshold)
    return inputAttributesMinGinis, attributeThresholds
   

In [ ]:
def createTree(trainIn,trainOut):
    attributesUsed = []
    return recursiveBranch(trainIn, trainOut, 1, attributesUsed, None)
    

def recursiveBranch(inputData, outputData, parentGini, attributesUsed, currentNode):

    if currentNode == None: # Make root node
        minAttributeGinis, attributeThresholds = getMinGiniAndThreshold(inputData, outputData, attributesUsed)
        nodeAttributeGini = min(minAttributeGinis)

        nodeAttributeIndex = minAttributeGinis.index(nodeAttributeGini)
        attributesUsed.append(nodeAttributeIndex)

        # print("Node attribute index (lowest gini) =",nodeAttributeIndex)

        nodeThreshold = attributeThresholds[minAttributeGinis.index(nodeAttributeGini)]
        # print("Root node created using attribute {} (has a min gini of {:.5f}). Threshold at this node = {}. Parent node gini = {:.5f}.".format(nodeAttributeIndex, nodeAttributeGini, nodeThreshold, parentGini))

        rootNode = Node(nodeAttributeIndex, nodeThreshold)
        branchData(inputData, outputData, nodeAttributeGini, nodeThreshold, nodeAttributeIndex, attributesUsed, rootNode)
        # print("Back to root")
        return rootNode


    leaf = False # Default
    # print(inputData.shape[0],"- Attributes Used =", attributesUsed)
    # print(getUniqueClassCount(list(outputData))[0],"unique classes")
    
    if inputData.shape[0] <= 1:
        # print("Just one sample remaining,",outputData)
        leaf = True
        
    if len(getUniqueClassCount(list(outputData))[0]) == 1:
        # print("Remaining {} samples are all {}.".format(inputData.shape[0], outputData[0]))
        leaf = True

    if leaf:
        currentNode.setValue(outputData)
        # print(outputData)
        return
    else: # Check ginis
        minAttributeGinis, attributeThresholds = getMinGiniAndThreshold(inputData, outputData, attributesUsed)
        nodeAttributeGini = min(minAttributeGinis)

        # If none of the new ginis is less than parent gini, exit recursion
        if parentGini < nodeAttributeGini:
            # print("Leaf created as min gini of remaining attributes = (gini = {:.5f}), but parent node is has gini = {:.5f}.".format(nodeAttributeGini, parentGini))
            currentNode.setValue(outputData)
            # print(outputData)
            return
            leaf = True

        else: # Not leaf -> Continue recursion
            attributeIndexes = list(getInputAttributeDict())
            for i in attributesUsed:
                attributeIndexes.remove(i)
            nodeAttributeIndex = attributeIndexes[minAttributeGinis.index(nodeAttributeGini)]
            attributesUsed.append(nodeAttributeIndex)

            nodeAttributeIndex = attributeIndexes[minAttributeGinis.index(nodeAttributeGini)]
            # print("Node attribute index (lowest gini) =",nodeAttributeIndex)

            nodeThreshold = attributeThresholds[minAttributeGinis.index(nodeAttributeGini)]
            # print("Branch node created using attribute {} (has a min gini of {:.5f}). Threshold at this node = {}. Parent node has gini = {:.5f}.".format(nodeAttributeIndex, nodeAttributeGini, nodeThreshold, parentGini))
            currentNode.setValue(nodeThreshold)
            currentNode.setIndex(nodeAttributeIndex)
            branchData(inputData, outputData, nodeAttributeGini, nodeThreshold, nodeAttributeIndex, attributesUsed, currentNode)
        return



In [ ]:
def branchData(X, y, splitAttributeGini, splitValue, splitAttributeIndex, attributesUsed, currentNode):
    lessThan_X = []
    lessThan_y = []
    greaterThan_X = []
    greaterThan_y = []

    for i in range(len(X)):
        if X[i,splitAttributeIndex] < splitValue:
            lessThan_X.append(X[i,:])
            lessThan_y.append(y[i])
        else:
            greaterThan_X.append(X[i,:])
            greaterThan_y.append(y[i])

    # left branch first:
    attributesUsed_left = attributesUsed.copy()
    attributesUsed_right = attributesUsed.copy()

    currentNode.addLeftChild(Node())
    currentNode.addRightChild(Node())

    # currentNode_right = currentNode.copy()
    
    lessThan_X = np.asarray(lessThan_X)
    lessThan_y = np.asarray(lessThan_y)
    # print("\nBranching left...")
    recursiveBranch(lessThan_X, lessThan_y, splitAttributeGini, attributesUsed_left, currentNode.getLeftChild())
    # print("Finished left branch.\n")

    

    # Right branch
    greaterThan_X = np.asarray(greaterThan_X)
    greaterThan_y = np.asarray(greaterThan_y)
    # print("\nBranching right...")
    recursiveBranch(greaterThan_X, greaterThan_y, splitAttributeGini, attributesUsed_right, currentNode.getRightChild())
    # print("Finished right branch.\n")

    return lessThan_X,lessThan_y,greaterThan_X,greaterThan_y
        

In [ ]:
class Node:
    def __init__(self,index=None, value=None):
        self.left = None
        self.right = None
        self.index = index
        self.value = value #stores value or the classification eg "ale"
        
    def addLeftChild(self,child):
        self.left = child
    
    def getLeftChild(self):
        return self.left

    def addRightChild(self,child):
        self.right = child

    def getRightChild(self):
        return self.right
    
    def printTree(self):
        if self.left:
            self.left.printTree()
        print(self.getData())
        if self.right:
            self.right.printTree()

    def getValue(self):
        return self.value

    def setValue(self, value):
        self.value = value

    def setIndex(self, index):
        self.index = index
    
    def getData(self):
        return self.index,self.value
    
    def isLeaf(self):
        if (self.left == None) & (self.right == None) & (self.index == None):
            return True
        else:
            return False

In [ ]:
root1 = createTree(trainIn,trainOut)

In [ ]:
def accuracy(yPredicted,yActual):
    correct = 0
    for i in range(len(yPredicted)):
        if (yPredicted[i] == yActual[i]):
            correct += 1
    return correct*100/len(yActual)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree
#forest 
rfcSklearn = RandomForestClassifier(100)
rfcSklearn = rfcSklearn.fit(trainIn, trainOut)
sklearnTrainPredictions = rfcSklearn.predict(trainIn)
sklearnTestPredictions = rfcSklearn.predict(testIn)
resultTraining = (sklearnTrainPredictions == trainOut )*100
resultTest = (sklearnTestPredictions == testOut)*100
# print("Performance using training data and sklearn","{:.4f}%".format(resultTraining.mean()))
# print("Performance using test data and sklearn","{:.4f}%".format(resultTest.mean()))

#tree
clfSklearn = tree.DecisionTreeClassifier()
clfSklearn = clfSklearn.fit(trainIn, trainOut)
sklearnTrainPrdictionsTree  = clfSklearn.predict(trainIn)
sklearnTestPrdictionsTree = clfSklearn.predict(testIn)
resultTestTree = (sklearnTestPrdictionsTree == testOut)*100
resultTrainingTree = (sklearnTrainPrdictionsTree == trainOut )*100
print("Performance using training data","{:.4f}%".format(resultTrainingTree.mean()))
print("Performance using test data","{:.4f}%".format(resultTestTree.mean()))



In [ ]:
def randomForestClassifierScratch(x,y,N):
    forest = []
    newX = []
    newY = []
    for i in range(0,N,1):
        newX,newY = bootstrapData(x,y)
        root1 = createTree(np.array(newX),np.array(newY))
        forest.append(root1)
    return np.array(forest)

def bootstrapData(X,Y):
    newX = []
    newY = []
    for j in range(len(X)):
        randIndex = np.random.randint(len(X))
        newX.append(X[randIndex])
        newY.append(Y[randIndex])
    return newX, newY

def predictForest(X,forest):
    predictions = []
    for i in range(len(X)):
        guess = []
        for tree in forest:
            guess.append(predictTree(X[i],tree))
        counts, classes = getUniqueClassCount(list(guess))
        majorityClass = classes[counts.index(max(counts))]
        predictions.append(majorityClass)
    return predictions


def predictTree(x,root):
    treePointer = root
    index,value = treePointer.getData()
    while treePointer.isLeaf() == False:
        if (x[index] < value):
            treePointer = treePointer.getLeftChild()
            index,value = treePointer.getData()
        else:
            treePointer = treePointer.getRightChild()
            index,value = treePointer.getData()

    counts, classes = getUniqueClassCount(list(value)) # get counts of classes in leaf
    majorityClass = classes[counts.index(max(counts))]

    return majorityClass




In [ ]:
forest = randomForestClassifierScratch(trainIn,trainOut,1)

In [ ]:
predicted = predictForest(trainIn,forest)
predicty = predictForest(testIn,forest)
print(predicty)
print(accuracy(predicty,testOut))

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
#Signle Tree Tests sklearn v scratch implementation
preformaceSklearn = []
preformaceScratch = []

for i in range(10):
    trainData, testData = splitData(dataset)
    xtestIn,yTestOut=separateInputOutput(testData)
    xtrainIn,yTrainOut = separateInputOutput(trainData)

    #Test tree script developed from scratch
    treeScratch = createTree(xtrainIn,yTrainOut) 
    predictedYScratch = []
    for i in range(len(xtestIn)):
        predictionScratch = predictTree(xtestIn[i],treeScratch)
        predictedYScratch.append(predictionScratch)
    preformaceScratch.append(accuracy(predictedYScratch,yTestOut))
    
    #Sklearn 
    treeSklearn = tree.DecisionTreeClassifier()
    treeSklearn = treeSklearn.fit(xtrainIn, yTrainOut)
    sklearnTestPrdictionsTree  = treeSklearn.predict(xtestIn)
    preformaceSklearn.append(accuracy(sklearnTestPrdictionsTree,yTestOut))

sklearn = mpatches.Patch(color='green', label='Scikit Learn Decision Tree Implementation')
scratch = mpatches.Patch(color='black', label='Decision Tree Implemented From Scratch')
plt.legend(handles=[sklearn,scratch])
plt.plot(preformaceScratch,c='black',marker = '.')
plt.plot(preformaceSklearn,c='green',ls = '--',marker = '.')
plt.show()

print("Average Performance using Scikit Learn Decision Tree Implementation","{:.4f}%".format(sum(preformaceSklearn)/len(preformaceSklearn)))
print("Average Performance using Decision Tree Implementation Made from Scratch","{:.4f}%".format(sum(preformaceScratch)/len(preformaceScratch)))